<h1> Proyecto Individual N°1 <h1>

<h3> Librerias que vamos a utilizar: </h3>

In [2]:
#Importamos las librerias que vamos a ir utilizando
import pandas as pd
from pandas import json_normalize
import numpy as np
import ast
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

<h3> Carga de Datasets: </h3>

In [3]:
#Cargamos los Datasets desde un archivos CSV con la libreria Pandas

peliculas = pd.read_csv('C:/Users/tomif/Desktop/Proyecto_1/Movies/movies_dataset.csv')
creditos = pd.read_csv('C:/Users/tomif/Desktop/Proyecto_1/Movies/credits.csv')

C:\Users\tomif\AppData\Local\Temp\ipykernel_11764\4016740141.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  peliculas = pd.read_csv('C:/Users/tomif/Desktop/Proyecto_1/Movies/movies_dataset.csv')


In [4]:
#Visualizamos el dataset de peliculas
peliculas.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
#Visualizamos el dataset de creditos
creditos.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [6]:
#Creamos df_credit_cast con las columnas "cast" y "id"
df_creditos_cast = creditos[['cast', 'id']].copy()

#Creamos df_credit_crew con las columnas "crew" y "id"
df_creditos_crew = creditos[['crew', 'id']].copy()

<h3> Comenzamos el ETL desanidando las columnas:<h3>

In [7]:
#Creamos una funcion que transforme las culumnas anidadas en columnas individuales con cada valor pretendiente.

def expandcolumns(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
        # Expandir la columna en filas
        df = df.explode(column)
        # Normalizar la columna expandida
        col_df = pd.json_normalize(df[column])
        # Renombrar las columnas normalizadas para evitar superposiciones
        col_df = col_df.add_prefix(f'{column}')
        # Concatenar las columnas normalizadas con el DataFrame original
        df = df.drop(columns=[column]).reset_index(drop=True).join(col_df)
    return df

In [8]:
#Desanidamos el dataset Peliculas
columns_to_expand_peliculas = ['genres', 'production_companies', 'production_countries', 'spoken_languages','belongs_to_collection']

#Expandimos las columnas de listas de diccionarios
df_peliculas= expandcolumns(peliculas, columns_to_expand_peliculas)
df_peliculas.head()

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,vote_average,vote_count,genresid,genresname,production_companiesname,production_companiesid,production_countriesiso_3166_1,production_countriesname,spoken_languagesiso_639_1,spoken_languagesname
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,7.7,5415.0,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English
1,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,7.7,5415.0,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English
2,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,7.7,5415.0,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English
3,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,7.7,5415.0,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English
4,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,7.7,5415.0,35.0,Comedy,Pixar Animation Studios,3.0,US,United States of America,en,English


In [9]:
#Expandimos las columnas de listas de diccionarios para otro df, asi resolvemos los duplicados mas adelante
df_peliculas_sin_id_duplicados= df_peliculas.drop_duplicates("id")
df_peliculas_sin_id_duplicados.head()

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,vote_average,vote_count,genresid,genresname,production_companiesname,production_companiesid,production_countriesiso_3166_1,production_countriesname,spoken_languagesiso_639_1,spoken_languagesname
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,7.7,5415.0,16.0,Animation,Pixar Animation Studios,3.0,US,United States of America,en,English
12,False,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,...,6.9,2413.0,12.0,Adventure,TriStar Pictures,559.0,US,United States of America,en,English
30,False,0,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,...,6.5,92.0,10749.0,Romance,Warner Bros.,6194.0,US,United States of America,en,English
46,False,16000000,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,...,6.1,34.0,35.0,Comedy,Twentieth Century Fox Film Corporation,306.0,US,United States of America,en,English
49,False,0,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,...,5.7,173.0,35.0,Comedy,Sandollar Productions,5842.0,US,United States of America,en,English


In [10]:
#Desanidamos el dataset Creditos_cast
columns_to_expand_creditos = ['cast']

#Expandimos las columnas de listas de diccionarios
df_creditos_cast= expandcolumns(df_creditos_cast, columns_to_expand_creditos)
df_creditos_cast.head()

,id,castcast_id,castcharacter,castcredit_id,castgender,castid,castname,castorder,castprofile_path
0,862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,862,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,862,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,862,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,862,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg


In [11]:
#Desanidamos el dataset Creditos_crew
columns_to_expand_creditos = ['crew']

#Expandimos las columnas de listas de diccionarios
df_creditos_crew= expandcolumns(df_creditos_crew, columns_to_expand_creditos)
df_creditos_crew.head()

,id,crewcredit_id,crewdepartment,crewgender,crewid,crewjob,crewname,crewprofile_path
0,862,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,862,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,862,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,862,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,862,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg


<h3> ETL(Transformacion y Limpieza de datos) del dataset Peliculas: </h3>

In [12]:
#Agrupar por el identificador único de la película y agregar listas de idiomas y géneros sin duplicados
df_movie_agg = df_peliculas_sin_id_duplicados.groupby('id').agg({
    'spoken_languagesname': lambda x: ', '.join(set(i for i in x if pd.notna(i))),  # Filtra nulos, elimina duplicados y une con "," los valores de spoken_languagesname
    'spoken_languagesiso_639_1': lambda x: ', '.join(set(i for i in x if pd.notna(i))),  # Filtra nulos, elimina duplicados y une con "," los valores de spoken_languagesname_639
    'genresid': lambda x: ', '.join(map(str, set(int(i) for i in x if pd.notna(i)))),  # Convierte a enteros, elimina duplicados y une los ID con coma
    'genresname': lambda x: ', '.join(set(i for i in x if pd.notna(i)))  # Eliminamos duplicados y unimos con coma
}).reset_index()

df_movie_agg.columns = ['id', 'spoken_languages_names', 'spoken_languages_iso', 'genres_ids', 'genres_names']

#Unimos los datos agregados de nuevo con el DataFrame
df_movie2 = df_peliculas_sin_id_duplicados.merge(df_movie_agg, on='id', how='left')

In [13]:
df_peliculas= df_movie2.drop_duplicates()

In [14]:
#Limpieza de datos duplicados del dataset df_creditos crew y cast

df_creditos_crew = df_creditos_crew.drop_duplicates()
df_creditos_cast = df_creditos_cast.drop_duplicates()

#Limpieza de datos duplicados del dataset df_peliculas

df_peliculas = df_peliculas.drop_duplicates()
df_peliculas_sin_id_duplicados = df_peliculas.drop_duplicates("id")

In [15]:
# Llenamos valores nulos en 'revenue' y 'budget' con 0
df_peliculas['revenue'].fillna(0, inplace=True)
df_peliculas['budget'].fillna(0, inplace=True)

C:\Users\tomif\AppData\Local\Temp\ipykernel_11764\226502065.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_peliculas['revenue'].fillna(0, inplace=True)
C:\Users\tomif\AppData\Local\Temp\ipykernel_11764\226502065.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exampl

In [16]:
# Eliminamos filas con valores nulos en 'release_date'
df_peliculas.dropna(subset=['release_date'], inplace=True)

In [17]:
# Convertimos 'release_date' a formato AAAA-mm-dd y crear 'release_year'
df_peliculas['release_date'] = pd.to_datetime(df_peliculas['release_date'], errors='coerce').dt.strftime('%Y-%m-%d')
df_peliculas['release_year'] = pd.to_datetime(df_peliculas['release_date'], errors='coerce').dt.year

# Eliminamos filas donde 'release_date' es nulo o no pudo convertirse
df_peliculas = df_peliculas.dropna(subset=['release_date'])

# Reiniciamos los índices después de la eliminación
df_peliculas.reset_index(drop=True, inplace=True)


In [18]:
#Al nosotros tenes una columna como object(str), tenemos que transformarla en float

# Convertimos las columnas revenue y budget a tipo numérico
df_peliculas['revenue'] = pd.to_numeric(df_peliculas['revenue'], errors='coerce')
df_peliculas['budget'] = pd.to_numeric(df_peliculas['budget'], errors='coerce')


# Calculamos el retorno de inversión (return) solo para las filas filtradas
df_peliculas['return'] = df_peliculas.apply(lambda row: row['revenue'] / row['budget'] if pd.notnull(row['revenue']) and pd.notnull(row['budget']) and row['budget'] != 0 else 0, axis=1)

# Agregamos filas eliminadas con valor 0 en 'return' y combinar con el DataFrame original
df_peliculas = pd.concat([df_peliculas, df_peliculas[df_peliculas.duplicated(subset=['revenue', 'budget'], keep=False)].fillna({'return': 0})])

# Reiniciamos los índices después de cualquier manipulación
df_peliculas.reset_index(drop=True, inplace=True)


In [19]:
# Esta es la lista de columnas a eliminar
columns_to_drop = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']

# Eliminamos las columnas especificadas
df_peliculas = df_peliculas.drop(columns=columns_to_drop)

In [20]:
df_peliculas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82915 entries, 0 to 82914
Data columns (total 27 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   budget                          82915 non-null  int64  
 1   id                              82915 non-null  object 
 2   original_language               82893 non-null  object 
 3   overview                        81069 non-null  object 
 4   popularity                      82915 non-null  object 
 5   release_date                    82915 non-null  object 
 6   revenue                         82915 non-null  float64
 7   runtime                         82432 non-null  float64
 8   status                          82757 non-null  object 
 9   tagline                         34669 non-null  object 
 10  title                           82915 non-null  object 
 11  vote_average                    82915 non-null  float64
 12  vote_count                      

<h3> ETL(Transformacion y Limpieza de datos) del dataset Creditos: </h3>

In [21]:
# Vamos a crear la columna Directores con los nombres de cada director

# Creamos una columna 'directores' con los nombres de los directores 
directores = df_creditos_crew[df_creditos_crew['crewjob'] == 'Director'].groupby('crewname')['id'].apply(list).reset_index()

# Renombramos las columnas adecuadamente
directores.columns = ['director', 'movies_directed']

# Fusionamos con el DataFrame original para agregar la nueva columna
df_creditos_crew = pd.merge(df_creditos_crew, directores, left_on='crewname', right_on='director', how='left')

df_creditos_crew = df_creditos_crew.drop(columns=['movies_directed'])



In [22]:
# Esta es la lista de columnas a eliminar en dataset crew
df_creditos_crew = df_creditos_crew.drop(columns=['crewcredit_id'])
df_creditos_crew = df_creditos_crew.drop(columns=['crewgender'])
df_creditos_crew = df_creditos_crew.drop(columns=['crewdepartment'])
df_creditos_crew = df_creditos_crew.drop(columns=['crewprofile_path'])
df_creditos_crew = df_creditos_crew.drop(columns=['crewjob'])
df_creditos_crew = df_creditos_crew.drop(columns=['crewname'])
df_creditos_crew = df_creditos_crew.drop(columns=['crewid'])

In [23]:
# Agrupar por la columna 'id' y combinar los actores en una lista
df_creditos_crew = df_creditos_crew.groupby('id')['director'].apply(lambda x: ', '.join(str(s) for s in x)).reset_index()

# Renombrar la columna resultante para mayor claridad
df_creditos_crew.columns = ['id', 'director']

df_creditos_crew.dropna(subset=['director'], inplace=True)

In [24]:
df_creditos_crew.head()

,id,director
0,2,"Aki Kaurismäki, Aki Kaurismäki, nan, nan, nan,..."
1,3,"Aki Kaurismäki, Aki Kaurismäki, nan, nan, nan,..."
2,5,"nan, nan, nan, Allison Anders, Alexandre Rockw..."
3,6,"Stephen Hopkins, nan, nan, nan, nan, nan, nan,..."
4,11,"George Lucas, George Lucas, George Lucas, nan,..."


In [25]:
# Vamos a crear la columna Actores con los nombres de cada actor

# Convertimos todos los valores de la columna 'castname' a cadenas de texto
df_creditos_cast['castname'] = df_creditos_cast['castname'].astype(str)

# Eliminamos filas con valores nulos en 'castname'
df_creditos_cast = df_creditos_cast.dropna(subset=['castname'])

# Eliminamos duplicados en la columna 'castname'
actores_sin_repetir = df_creditos_cast.drop_duplicates(subset=['castname'])

# Agrupamos por 'id' y combinar los nombres de los actores en una lista
actores_agrupados = actores_sin_repetir.groupby('id')['castname'].apply(lambda x: ', '.join(x)).reset_index()

# Renombramos la columna resultante
actores_agrupados.columns = ['id', 'actores']

# Fusionamos con el DataFrame original para agregar la nueva columna
df_creditos = pd.merge(df_creditos_cast, actores_agrupados, on='id', how='left')


In [26]:
# Esta es la lista de columnas a eliminar en dataset cast
df_creditos_cast = df_creditos_cast.drop(columns=['castcast_id'])
df_creditos_cast = df_creditos_cast.drop(columns=['castcharacter'])
df_creditos_cast = df_creditos_cast.drop(columns=['castcredit_id'])
df_creditos_cast = df_creditos_cast.drop(columns=['castgender'])
df_creditos_cast = df_creditos_cast.drop(columns=['castid'])
df_creditos_cast = df_creditos_cast.drop(columns=['castorder'])
df_creditos_cast = df_creditos_cast.drop(columns=['castprofile_path'])

In [27]:
# Agrupar por la columna 'id' y combinar los actores en una lista
df_creditos_cast = df_creditos_cast.groupby('id')['castname'].apply(lambda x: ', '.join(x)).reset_index()

# Renombrar la columna resultante para mayor claridad
df_creditos_cast.columns = ['id', 'actores']

In [28]:
df_creditos_cast.head()

,id,actores
0,2,"Turo Pajala, Susanna Haavisto, Matti Pellonpää..."
1,3,"Matti Pellonpää, Kati Outinen, Sakari Kuosmane..."
2,5,"Tim Roth, Antonio Banderas, Jennifer Beals, Ma..."
3,6,"Emilio Estevez, Cuba Gooding Jr., Denis Leary,..."
4,11,"Mark Hamill, Harrison Ford, Carrie Fisher, Pet..."


<h1> FastAPI</h1>

In [29]:
from fastapi import FastAPI

app = FastAPI()

<h3>ETL(Extraccion mediante funciones):

In [30]:
@app.get("/Meses")

# Función para contar la cantidad de películas estrenadas en un mes específico
def cantidad_filmaciones_mes(mes):
    # Convertimos la columna 'release_date' a tipo datetime si no está en ese formato
    df_peliculas['release_date'] = pd.to_datetime(df_peliculas['release_date'], errors='coerce')
    
    # Filtramos películas que coincidan con el mes dado
    peliculas_en_mes = df_peliculas[df_peliculas['release_date'].dt.month_name('es').str.lower() == mes.lower()]
    
    # Eliminamos duplicados basados en la columna 'movie_title' por ejemplo, ajusta según tus datos
    peliculas_en_mes_sin_duplicados = peliculas_en_mes.drop_duplicates(subset=['title'])
    
    # Contamos la cantidad de películas únicas en el mes
    cantidad = peliculas_en_mes_sin_duplicados.shape[0]
    
    return cantidad

# Ejemplo de uso de la función ingresando directamente el nombre del mes
cantidad = cantidad_filmaciones_mes('Febrero')
print(f"En el mes de Febrero, se estrenaron {cantidad} películas únicas.")

En el mes de Febrero, se estrenaron 3014 películas únicas.


In [31]:
@app.get("/Dias")
def cantidad_filmaciones_dia(Dia):

#Mapeo de nombres de días en español a números de día. Se incluyen variantes sin acento ortográfico.
    dias = {'lunes': 0, 'martes': 1, 'miércoles': 2, 'miercoles': 2, 'jueves': 3, 'viernes': 4, 'sábado': 5, 'sabado': 5,'domingo': 6}

#Convertimos el día ingresado a minúsculas y obtener su número correspondiente

    dia_numero = dias.get(Dia.lower(), None)

    if dia_numero is None:
        return f"No se reconoce el día '{Dia}'. Por favor, ingresa un día válido en español."

#Filtramos las películas por el día especificado
    peliculas_dia = df_peliculas[df_peliculas['release_date'].dt.dayofweek == dia_numero]

# Eliminamos duplicados basados en la columna 'movie_title' por ejemplo, ajusta según tus datos
    peliculas_en_dia = peliculas_dia.drop_duplicates(subset=['title'])
    

#Obtenemos la cantidad de películas
    cantidad = len(peliculas_en_dia)

#Devolvemos el resultado formateado
    return f"{cantidad} {'películas' if cantidad != 1 else 'película'} fueron estrenadas en los días {Dia.title()}."

# Ejemplo de uso de la función ingresando directamente el nombre del día
cantidad = cantidad_filmaciones_dia('lunes')
print(cantidad)

3471 películas fueron estrenadas en los días Lunes.


In [32]:
@app.get("/Titulos")

# Función para obtener el título, año de estreno y score de una filmación
def score_titulo(titulo):

    # Filtramos el DataFrame por el título dado
    pelicula = df_peliculas[df_peliculas['title'] == titulo]
    
    # Verificaamos si se encontró la película
    if pelicula.empty:
        return f"No se encontró información para '{titulo}'"
    
    # Obtenemos los datos de la película encontrada
    titulo = pelicula['title'].iloc[0]
    año_estreno = pelicula['release_year'].iloc[0]
    score = pelicula['popularity'].iloc[0]
    
    return titulo, año_estreno, score

# Ejemplo de uso de la función con un título específico ingresado directamente
resultado = score_titulo('Toy Story')
if isinstance(resultado, tuple):
    print(f"Para la película '{resultado[0]}':")
    print(f"Título: {resultado[0]}")
    print(f"Año de estreno: {resultado[1]}")
    print(f"Score: {resultado[2]}")
else:
    print(resultado)

Para la película 'Toy Story':
Título: Toy Story
Año de estreno: 1995.0
Score: 21.946943


In [33]:
@app.get("/Votos")
# Función para obtener el título, cantidad de votos y valor promedio de votaciones
def votos_titulo(titulo):
    # Filtramos el DataFrame por el título dado
    pelicula = df_peliculas[df_peliculas['title'] == titulo]
    
    # Verificamos si se encontró la película y si cumple con la condición de votos
    if pelicula.empty:
        return f"No se encontró información para '{titulo}'"
    
    if pelicula['vote_count'].iloc[0] < 2000:
        return f"La película '{titulo}' no cumple con el mínimo de 2000 valoraciones."
    
    # Obtenemos los datos de la película encontrada
    titulo = pelicula['title'].iloc[0]
    votos = pelicula['vote_count'].iloc[0]
    promedio_votos = pelicula['vote_average'].iloc[0]
    
    return "El tirulo es: " +str(titulo)+ " / Los votos que tuvo fueron: " +str(votos)+ " / Y el promedio de votos es : " +str(promedio_votos)

# Ejemplo de uso de la función con un título específico ingresado directamente
resultado = votos_titulo('Toy Story')
if isinstance(resultado, tuple):
    print(f"Para la película '{resultado[0]}':")
    print(f"Título: {resultado[0]}")
    print(f"Cantidad de votos: {resultado[1]}")
    print(f"Valor promedio de votos: {resultado[2]}")
else:
    print(resultado)

El tirulo es: Toy Story / Los votos que tuvo fueron: 5415.0 / Y el promedio de votos es : 7.7


In [34]:
# Convertimos 'id' a tipo int si no coinciden
df_peliculas['id'] = df_peliculas['id'].astype(int)

df_creditos_crew['id'] = df_creditos_crew['id'].astype(int)

In [35]:
@app.get("/Actores")

# Función para obtener información de un actor específico
def get_actor(nombre_actor):

    # Filtramos las películas en las que ha participado el actor
    peliculas_actor = df_creditos_cast[df_creditos_cast['actores'].str.contains(nombre_actor, na=False)]
    
    # Eliminamos duplicados en 'castname' para contar películas únicas
    peliculas_actor = peliculas_actor.drop_duplicates(subset=['actores'])
    
    # Unimos con el DataFrame de películas para obtener la información de retorno
    peliculas_actor = pd.merge(peliculas_actor, df_peliculas[['id', 'return']], on='id', how='left')
    
    # Calculamos la cantidad de películas en las que ha participado el actor (sin duplicados)
    cantidad_peliculas = peliculas_actor.shape[0]
    
    # Calculamos el éxito total (suma de retornos)
    exito_total = peliculas_actor['return'].sum()
    
    # Calculamos el promedio de retorno
    promedio_retorno = peliculas_actor['return'].mean()
    
    return 'Nombre del actor: ' +str(nombre_actor)+ '/ Cantidad de peliculas grabadas: ' +str(cantidad_peliculas)+ '/ Exito total: ' +str(exito_total)+ '/ Promedio del retorno: ' +str(promedio_retorno)

# Ejemplo de uso
resultado = get_actor('Tom Hanks')
print(resultado)

Nombre del actor: Tom Hanks/ Cantidad de peliculas grabadas: 96/ Exito total: 178.8494960552024/ Promedio del retorno: 1.8630155839083582


In [36]:
@app.get("/Directores")

def get_director(nombre_director):

    # Filtramos las películas dirigidas por el director
    peliculas_director = df_creditos_crew[df_creditos_crew['director'].str.contains(nombre_director, na=False)]

    # Eliminamos duplicados en 'director' para contar películas únicas
    peliculas_director = peliculas_director.drop_duplicates(subset=['director'])
    
    # Unimos con el DataFrame de películas para obtener más detalles
    peliculas_director = pd.merge(peliculas_director, df_peliculas[['title', 'release_date', 'return', 'budget', 'revenue', 'id']], 
                                  left_on='id', right_on='id', how='left')
    
    # Calculamos el éxito total del director (suma de retornos)
    exito_total = peliculas_director['return'].sum()
    
    # Preparamos la lista de detalles de cada película única
    detalles_peliculas = []
    peliculas_procesadas = set()  # Conjunto para almacenar ids de películas procesadas
    
    for index, pelicula in peliculas_director.iterrows():
        movie_id = pelicula['id']
        # Verificamos si ya procesamos esta película (evitamos duplicados)
        if movie_id not in peliculas_procesadas:
            detalles_peliculas.append({
                'nombre_pelicula': pelicula['title'],
                'fecha_lanzamiento': pelicula['release_date'].strftime('%Y-%m-%d'),
                'retorno_individual': pelicula['return'],
                'costo': pelicula['budget'],
                'ganancia': pelicula['revenue']
            })
            peliculas_procesadas.add(movie_id)
    
    # Preparamos el mensaje final
    mensaje = (
        f"Director: {nombre_director}\n"
        f"Éxito total: {exito_total}\n"
        f"Películas:\n"
    )
    
    # Imprimimos los detalles de las películas de manera organizada
    for detalle in detalles_peliculas:
        mensaje += (
            f"  - {detalle['nombre_pelicula']} ({detalle['fecha_lanzamiento']}): "
            f"Retorno {detalle['retorno_individual']}, Costo {detalle['costo']}, Ganancia {detalle['ganancia']}\n"
        )
    
    # Imprimimos la información para la lectura
    print(mensaje)
    
    

# Ejemplo de uso
resultado = get_director('John Lasseter')
print(resultado)

Director: John Lasseter
Éxito total: 145.9549392653437
Películas:
  - Finding Nemo (2003-05-30): Retorno 10.003569531914893, Costo 94000000, Ganancia 940335536.0
  - Monsters, Inc. (2001-11-01): Retorno 4.8940544, Costo 115000000, Ganancia 562816256.0
  - Toy Story (1995-10-30): Retorno 12.4518011, Costo 30000000, Ganancia 373554033.0
  - Toy Story 2 (1999-10-30): Retorno 5.526298544444445, Costo 90000000, Ganancia 497366869.0
  - Cars (2006-06-08): Retorno 3.849859575, Costo 120000000, Ganancia 461983149.0
  - Ratatouille (2007-06-22): Retorno 4.15815212, Costo 150000000, Ganancia 623722818.0
  - Howl's Moving Castle (2004-11-19): Retorno 9.779602291666667, Costo 24000000, Ganancia 234710455.0
  - A Bug's Life (1998-11-25): Retorno 3.0271571583333334, Costo 120000000, Ganancia 363258859.0
  - The Incredibles (2004-11-05): Retorno 6.863501, Costo 92000000, Ganancia 631442092.0
  - Beauty and the Beast (1991-11-13): Retorno 15.09402212, Costo 25000000, Ganancia 377350553.0
  - Toy Story

<h2> Extraccion de datasets </h2>

In [37]:
# Unimos todos los datasets en un solo archivo CSV

# Primero, combinamos `peliculas` con `actores` usando merge
df_main = pd.merge(df_peliculas, df_creditos_cast, on='id', how='left')

# Despues, combinamos el resultado con `directores` usando merge
df_main = pd.merge(df_main, df_creditos_crew, on='id', how='left')

In [38]:
df_main = df_main.drop_duplicates()

In [43]:
# Por ultimo guardamos los DataFrame en un archivo CSV y dejar un los datasets divididos por si surgen errores mas adelante

##df_creditos_cast.to_csv('creditos_cast.csv', index=False)

##df_creditos_crew.to_csv('creditos_crew.csv', index=False)

##df_peliculas.to_csv('peliculas.csv', index=False)

##df_main.to_csv('Main.csv', index=False)

In [40]:
df_main.head()

<class 'pandas.core.frame.DataFrame'>
Index: 45346 entries, 0 to 45345
Data columns (total 29 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   budget                          45346 non-null  int64         
 1   id                              45346 non-null  int32         
 2   original_language               45335 non-null  object        
 3   overview                        44405 non-null  object        
 4   popularity                      45346 non-null  object        
 5   release_date                    45346 non-null  datetime64[ns]
 6   revenue                         45346 non-null  float64       
 7   runtime                         45100 non-null  float64       
 8   status                          45266 non-null  object        
 9   tagline                         20387 non-null  object        
 10  title                           45346 non-null  object        
 11  vote_av

In [41]:
df_main.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,spoken_languagesiso_639_1,spoken_languagesname,spoken_languages_names,spoken_languages_iso,genres_ids,genres_names,release_year,return,actores,director
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,en,English,English,en,16,Animation,1995.0,12.451801,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...","John Lasseter, Joss Whedon, Andrew Stanton, na..."
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,en,English,English,en,12,Adventure,1995.0,4.043035,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...","nan, Jonathan Hensleigh, nan, Joe Johnston, Ro..."
2,0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,en,English,English,en,10749,Romance,1995.0,0.000000,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...","Howard Deutch, Mark Steven Johnson, Mark Steve..."
3,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,...,en,English,English,en,35,Comedy,1995.0,5.090760,"Whitney Houston, Angela Bassett, Loretta Devin...","Forest Whitaker, nan, nan, nan, nan, nan, nan,..."
4,0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,...,en,English,English,en,35,Comedy,1995.0,0.000000,"Steve Martin, Diane Keaton, Martin Short, Kimb...","nan, nan, Nancy Meyers, Nancy Meyers, nan, Cha..."
